In [14]:
import os
os.environ['PYTHONHASHSEED'] = '1'
from numpy.random import seed
seed(1)
import random as rn
rn.seed(1)

import warnings
warnings.filterwarnings("ignore")


import sys
import glob
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib import cm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.decomposition import PCA
from scipy.stats import kurtosis 
from scipy.stats import skew
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from collections import Counter
from sklearn import metrics
from sklearn.feature_selection import f_classif
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import GradientBoostingClassifier
#from yellowbrick.features.importances import FeatureImportances
from sklearn.model_selection import learning_curve
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from sklearn.pipeline import Pipeline  
import matplotlib.style as style 
from sklearn.pipeline import Pipeline       
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import ast
import gc
from numpy import sqrt
from numpy import argmax
import numpy as np
from sklearn.preprocessing import LabelBinarizer
np.set_printoptions(threshold=sys.maxsize)

from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scipy import interp
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn import preprocessing




In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Data Variables

- root: Select the folder name of your dataset
- initial component list, number_initial_components: Create the list containing the available signals and the number of them you would like to initially consider
- best_features_number: Select the máximun number of features to process (in this case 486 for the 9 initial signals)
- Dnumber_of_segments: Determine the number of subwinsdows in which every data example will be segmented
- n_splits: the number of folds in the cross validation 
- n_times: constant to determine how many times results will be obtained and averaged


In [3]:
#Dataset folder relative route
root = './OHM_Dataset/' 

"""
 Available signals: acc x,y,z 
                    gyro x,y,z , 
                    pitch roll and yaw
"""

initial_componet_list=["x", "y", "z", "gyrox", "gyroy", "gyroz", "pitch","roll","yaw"]

#In case you want to reduce the number of signals 
number_initial_components = 9
initial_data_reduced_names = initial_componet_list[0:number_initial_components ]

#Best feature number to process 
#(Max 486 for 9 Signals)
best_features_number=486

#Number of subwindos in which every window is divided
number_of_segments=5

#Number in X-fold Cross Validation
n_splits=5

#Repeat the cross validation n_times and average final results
n_times = 10


# Model Cascade Configuration

- Log_loss_T, log_loss_T_sel: Set the confidence threshold according to its log loss value for model 1-2 and model 2-3 of the cascade

- n_features_mN: Select the number of features used in every model N

- segments_mN: Determine wheter the models are created using the segmented data or not

In [4]:
#Threshold 1
log_loss_T=0.15
#Threshold 2
log_loss_T_sel=0.4


#Determine wheter the classification will rely on the segmented instances or not
#segments True= segmened 
#segments False = NO segmented


#----------------model 1
n_features_m1=10 #number of selected features for model 1
segments_m1=True

#----------------model 2
n_features_m2=50 #number of selected features for model 2
segments_m2=True



#----------------model 3 (Final)
n_features_m3=486 #(all features)
segments_m3=True


# Data processing


In [5]:

#Create activities dictionary 
wrist_class = {'Other':0, 
              'Drink_glass':1, 
               'Drink_bottle':2, 
             }

n=0
best_features_num=[best_features_number]

#Activity label list
wrist_labels = sorted(wrist_class, key=lambda x: x[1], reverse=True)
labels=wrist_labels
number_classes=len(wrist_labels)
print(wrist_labels)


#Funtion to import all the txt for every class, calculate its features and append it in a Matrix
def gather(class_dict,split_index,segments):
    df = []
    for c in class_dict.keys():
        f = glob.glob(root + c + '/*') #Get all files 
        d = pd.DataFrame(reformat(f, cls=c,split_index=split_index, segments=segments)) #Pandas dataframe for reformat funtion features
        df.append(d)
    return pd.concat(df)


#Median Filter axiliar function 
def strided_app(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S*n,n)) 



#Obtain Data values for every secuence/instance
def reformat(files, cls,  split_index, segments):
    big_list= []
    for f in files:
        #Read every txt file (Number of row value_x, Value_Y, Value_Z)
        data = pd.read_csv(f, sep=',', header=None, names=['x', 'y', 'z', 'gyrox' , 'gyroy', 'gyroz', 'pitch', 'roll', 'yaw']) 

        
        df_x = data.iloc[:,0:1]
        df_y = data.iloc[:,1:2]
        df_z = data.iloc[:,2:3]
    
        df_gyrox = data.iloc[:,3:4]
        df_gyroy = data.iloc[:,4:5]
        df_gyroz = data.iloc[:,5:6]
        
        df_pitch = data.iloc[:,6:7]
        df_roll = data.iloc[:,7:8]
        df_yaw = data.iloc[:,8:9]
        
        
        
        #Median filtering
        x = np.median(strided_app(df_x.values.flatten(), 3,1),axis=1)
        y = np.median(strided_app(df_y.values.flatten(), 3,1),axis=1)
        z = np.median(strided_app(df_z.values.flatten(), 3,1),axis=1)
        
        gyrox = np.median(strided_app(df_gyrox.values.flatten(), 3,1),axis=1)
        gyroy = np.median(strided_app(df_gyroy.values.flatten(), 3,1),axis=1)
        gyroz = np.median(strided_app(df_gyroz.values.flatten(), 3,1),axis=1)
        
        pitch = np.median(strided_app(df_pitch.values.flatten(), 3,1),axis=1)
        roll = np.median(strided_app(df_roll.values.flatten(), 3,1),axis=1)
        yaw = np.median(strided_app(df_yaw.values.flatten(), 3,1),axis=1)
        
        
        
        
        data_all=pd.concat([df_x.reset_index(drop=True), df_y.reset_index(drop=True), df_z.reset_index(drop=True),
                           df_gyrox.reset_index(drop=True), df_gyroy.reset_index(drop=True), df_gyroz.reset_index(drop=True) ,
                           df_pitch.reset_index(drop=True), df_roll.reset_index(drop=True), df_yaw.reset_index(drop=True),
                           ],
                           
                           axis=1)
        
        
        data_reduced = data_all.iloc[:,0:number_initial_components ]
        
        
        #Split selected component of every secuence in X segments
        split_index=split_index #Number of segments
        data_split=np.array_split(data_reduced, split_index)
        
        appended_features=[]
        
        #Features calculation
        features_whole = pd.concat([data_reduced.mean(axis=0).rename(index=lambda x: 'mean' + '_' + x), 
                              #Dataframe 
                              data_reduced.std(axis=0).rename(index=lambda x: 'std' + '_' + x),
                              data_reduced.median(axis=0).rename(index=lambda x: 'median' + '_' + x), 
                              data_reduced.mad(axis=0).rename(index=lambda x: 'mad' + '_' + x), 
                              data_reduced.max(axis=0).rename(index=lambda x: 'max' + '_' + x),
                              data_reduced.kurtosis(axis=0).rename(index=lambda x: 'kur' + '_' + x),  
                              data_reduced.skew(axis=0).rename(index=lambda x: 'skw' + '_' + x), 
                              data_reduced.var(axis=0).rename(index=lambda x: 'var' + '_' + x),   
                              data_reduced.min(axis=0).rename(index=lambda x: 'min' + '_' + x)])
        
        for i in range(0, split_index):
            #Features for every segment of data
            features= pd.concat([data_split[i].mean(axis=0).rename(index=lambda x: 'mean' + '_' + x + '_' + str(i)), 
                              data_split[i].std(axis=0).rename(index=lambda x: 'std' + '_' + x + '_' + str(i)),
                              data_split[i].median(axis=0).rename(index=lambda x: 'median' + '_' + x + '_' + str(i)), 
                              data_split[i].mad(axis=0).rename(index=lambda x: 'mad' + '_' + x + '_' + str(i)), 
                              data_split[i].max(axis=0).rename(index=lambda x: 'max' + '_' + x + '_' + str(i)),
                              data_split[i].kurtosis(axis=0).rename(index=lambda x: 'kur' + '_' + x + '_' + str(i)),  
                              data_split[i].skew(axis=0).rename(index=lambda x: 'skw' + '_' + x + '_' + str(i)),
                              #data_split[i].sum(axis=0).rename(index=lambda x: 'sum' + '_' + x + '_' + str(i)), 
                              data_split[i].var(axis=0).rename(index=lambda x: 'var' + '_' + x + '_' + str(i)),
                              data_split[i].min(axis=0).rename(index=lambda x: 'min' + '_' + x + '_' + str(i))])
            appended_features.append(features)
        
        #Concat all obtained features
        if segments is False:
             appended_features_all = pd.concat([features_whole])
             
        else:
            #Concat all obtained features
             
            appended_features_all = pd.concat([features_whole])
            for i in range (0, split_index):
                appended_features_all = pd.concat([appended_features_all, appended_features[i]])
                    
        

        
        #Access to dictionary class number and add it as a feature
        appended_features_all['Y'] = wrist_class[cls]
        big_list.append(appended_features_all)

    #Return table containing all rows for every class and feature colums 
    #(mean*3, sd*3, Max*3, Min*3, Y). Number of the row is manteined. (0~101)
    return big_list



['Other', 'Drink_glass', 'Drink_bottle']


#  Data processing with subwindows

when segments = True

Each sequence of data is segmented in 5 subwindows and features all calculated for each of the 5 segments

In [6]:
#%%timeit 
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199


number_of_segments=n_splits

#Check if the input for n_splits is is greater than 1(signal is divided in n_splits ) or not (signal is not divided)
if number_of_segments <= 1:
    segments=False #No segmentation is done
else:
    segments=True #Signal divided in n segments
    
    
#Feature table for all classes
wrist_df = gather(wrist_class,segments=segments,split_index=number_of_segments)

#Extact number of the class (feature Y)
wrist_Y = np.asarray(wrist_df.Y)
n_samples=len(wrist_Y)

#Total number of features
num_features=len(wrist_df.columns)-1
print ('Total features: {}'.format(num_features))
#Extact rest of colums (features)
wrist_X = np.asarray(wrist_df.iloc[:,:num_features])
wrist_X_df = pd.DataFrame(wrist_X, columns=(wrist_df.iloc[:,:num_features]).columns)

#Maxium number of available features
all_features = num_features 
print ("Feature names  " + str(wrist_df.iloc[:,:num_features].columns.values.tolist()))

Total features: 486
Feature names  ['mean_x', 'mean_y', 'mean_z', 'mean_gyrox', 'mean_gyroy', 'mean_gyroz', 'mean_pitch', 'mean_roll', 'mean_yaw', 'std_x', 'std_y', 'std_z', 'std_gyrox', 'std_gyroy', 'std_gyroz', 'std_pitch', 'std_roll', 'std_yaw', 'median_x', 'median_y', 'median_z', 'median_gyrox', 'median_gyroy', 'median_gyroz', 'median_pitch', 'median_roll', 'median_yaw', 'mad_x', 'mad_y', 'mad_z', 'mad_gyrox', 'mad_gyroy', 'mad_gyroz', 'mad_pitch', 'mad_roll', 'mad_yaw', 'max_x', 'max_y', 'max_z', 'max_gyrox', 'max_gyroy', 'max_gyroz', 'max_pitch', 'max_roll', 'max_yaw', 'kur_x', 'kur_y', 'kur_z', 'kur_gyrox', 'kur_gyroy', 'kur_gyroz', 'kur_pitch', 'kur_roll', 'kur_yaw', 'skw_x', 'skw_y', 'skw_z', 'skw_gyrox', 'skw_gyroy', 'skw_gyroz', 'skw_pitch', 'skw_roll', 'skw_yaw', 'var_x', 'var_y', 'var_z', 'var_gyrox', 'var_gyroy', 'var_gyroz', 'var_pitch', 'var_roll', 'var_yaw', 'min_x', 'min_y', 'min_z', 'min_gyrox', 'min_gyroy', 'min_gyroz', 'min_pitch', 'min_roll', 'min_yaw', 'mean_x_0'

#  Data processing without subwindows

when segments = False

Each sequence of data is considered as a unique window. Features are calculated for that window.

# Algorithms

Run the corresponding cell to perform the analysis based on the corresponding classifier

### Logistic Regression (LG)

In [7]:
#LG
n=0
alg_array = [LogisticRegression(random_state=n)] 
alg_array_names  = ['LG']
item_name="LG"
item=LogisticRegression(random_state=n)



### Random Forest  (RF)

In [19]:
#RF
n=0
alg_array = [RandomForestClassifier(verbose= 0,n_estimators= 100,random_state= n)]
alg_array_names  = ['RF']
item_name="RF"
item=RandomForestClassifier(verbose= 0,n_estimators= 100,random_state= n)

RandomForestClassifier(random_state=0)


### K-Nearest Neighbour (KNN)

In [26]:
#KNN -> AÑADIDOS MÁS N
n=0
alg_array = [KNeighborsClassifier(n_neighbors=3)]
alg_array_names  = ['KNN']
item_name="KNN"
item=KNeighborsClassifier(n_neighbors=3)
             
             

### Naive Bayes (NB)

In [35]:
#NB
n=0
alg_array = [ GaussianNB()]
alg_array_names  = ['NB']
item_name="NB"
item=GaussianNB()


### Support Vector Machines (SVM)

In [50]:
#SVM
n=0
alg_array = [svm.SVC(kernel='linear', C=64.0, random_state=n, probability=True)]
alg_array_names  = ['SVM']
item_name="SVM"
item=svm.SVC(kernel='linear', C=64.0, random_state=n, probability=True)

### Multilayer perceptron (MP)

In [58]:
#MLP
n=0
alg_array = [MLPClassifier(hidden_layer_sizes=(16, 16, 16), max_iter=1000, random_state=n)]
alg_array_names  = ['MLP']
item_name="MLP"
item=MLPClassifier(hidden_layer_sizes=(16, 16, 16), max_iter=1000, random_state=n)

# Refence classfier

Reference results for the more complex model (trained using all the available features)

In [10]:
ref_score=[]
scoring = ['recall_macro','precision_macro','f1_macro','accuracy']


for n_features in best_features_num:
    select_feature = SelectKBest(chi2, k=n_features)
    print ('----------------------------- ')
    print ('---Number of Features %0.0f-----' % n_features)
    print ('----------------------------- ')
    print('\n')

    for item, item_name in zip(alg_array, alg_array_names): 
        print ('########### ' + str(item_name) + ' ##########\n')
        for score in scoring:
            pipe = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
                             ('Feature selection',select_feature), ('Algorithm',item)])

            for n in range(0,n_times):
                kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=n)
                scores = cross_val_score(pipe, wrist_X, wrist_Y, cv =kf, scoring = score)
                features = pipe.named_steps['Feature selection']
                average_score = (np.mean(scores)*100)
                ref_score.append(average_score)
            
            
            #print (array_dict_pre[item_name])
            #print (str(score) + ' - ' + ' MEAN  '  + str(np.mean(array_dict_pre[item_name])) + ' - SD '+ str(np.std(array_dict_pre[item_name])))
            #print (str(score) + ' - ' + ' MEAN  ' + '%.2f '  + ' - SD ' +  '%.3f ' % (np.mean(array_dict_pre[item_name]), np.std(array_dict_pre[item_name])))
            print(str(score) + ' - ' '%.2f (SD %.3f)' % (np.mean(ref_score), np.std(ref_score)))
            ref_score=[]
        
        print('\n')



----------------------------- 
---Number of Features 486-----
----------------------------- 


LogisticRegression(random_state=0)
########### LG ##########
recall_macro - 87.79 (SD 0.744)
precision_macro - 87.79 (SD 0.628)
f1_macro - 87.69 (SD 0.708)
accuracy - 89.37 (SD 0.524)




# Model Ensemble techniques

## Stacking

In [13]:

results_n_times=[]

print("Staking\n")
print("ALG   "+str(item)+"\n")
for n in range(0,n_times):
    kf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=n)    
    select_feature_1 = SelectKBest(chi2, k=n_features_m1)
    select_feature_2 = SelectKBest(chi2, k=n_features_m2)
    select_feature_3 = SelectKBest(chi2, k=n_features_m3)
    
    #Definition of the models that constitutes the stacking ensemble
    estimators = [
    ('10_features', Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
                ('Feature selection',select_feature_1),
                                ('Algorithm',item)])),
    ('50_features', Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
                ('Feature selection',select_feature_2),
                                ('Algorithm',item)])),
    ('486_features', Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
                ('Feature selection',select_feature_3),
                                ('Algorithm',item)]))]

    clf = StackingClassifier(estimators=estimators, final_estimator=item, n_jobs=-1)
    pred=cross_val_predict(clf, wrist_X, wrist_Y, cv =kf)
    results_1 = cross_val_score(clf, wrist_X, wrist_Y, cv=kf, scoring = "f1_macro")
    results_n_times.append(results_1.mean())



print('%.2f (SD %.3f)' % (np.mean(results_n_times)*100, np.std(results_n_times)*100))




Staking

ALG   LogisticRegression(random_state=0)

87.58 (SD 0.900)


# MODEL CASCADE

## Functions definition

In [46]:
values_classes=np.unique(wrist_Y, return_index=False, return_inverse=False, return_counts=False, axis=None)
start=values_classes[0]

#Multiclass log loss
def multiclass_log_loss(y_true, y_pred,eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1)

#Check the definition of the segments statement and select the initial feature matrix accordingly
def check_segments(segments):
    if segments==True:
        X=wrist_X #Feature matrix with segments
    else:
        X=wrist_X_NO_S #Feature matrix whitout segments
    return X


#Check if any element is missing in the probabilities of prediction array to avoid errors
def missing_elements(L, labels, Y):
    global start
    #start, end = Y[0], L[-1]
    end = Y[0]
    if start==0:
        sort=sorted(set(range(0, len(labels))).difference(L))
        for i in sort:
            if i == end:
                i=0
    else:
        sort=sorted(set(range(1, len(labels)+ 1)).difference(L))
    return sort


#Chech if the number of remaining instances is enough to perform a 5-CV 
def check_CV_instances(Y):
    unique, counts = np.unique(Y, return_counts=True)
    j=0
    return_=False
    for i in counts:
        if i == 1:
            j=j+1
            if j == 2:
                return_ = True
    return return_

import numpy as np
from sklearn.preprocessing import LabelBinarizer


#Class to binarize the obtained prediction value to have a binary array instead of an integer value
class LabelBinarizer2:
    def __init__(self):
        self.lb = LabelBinarizer()
    def fit(self, X):
        # Convert X to array
        X = np.array(X)
        # Fit X using the LabelBinarizer object
        self.lb.fit(X)
        # Save the classes
        self.classes_ = self.lb.classes_

    def fit_transform(self, X):
        # Convert X to array
        X = np.array(X)
        # Fit + transform X using the LabelBinarizer object
        Xlb = self.lb.fit_transform(X)
        # Save the classes
        self.classes_ = self.lb.classes_
        if len(self.classes_) == 2:
            Xlb = np.hstack((Xlb, 1 - Xlb))
        return Xlb

    def transform(self, X):
        # Convert X to array
        X = np.array(X)
        # Transform X using the LabelBinarizer object
        Xlb = self.lb.transform(X)
        if len(self.classes_) == 2:
            Xlb = np.hstack((Xlb, 1 - Xlb))
        return Xlb

    def inverse_transform(self, Xlb):
        # Convert Xlb to array
        Xlb = np.array(Xlb)
        if len(self.classes_) == 2:
            X = self.lb.inverse_transform(Xlb[:, 0])
        else:
            X = self.lb.inverse_transform(Xlb)
        return X
            
        
#Fill the gaps in incomplete arrays
def complete_arrays (Y,pred,pred_binarized,proba,labels):
    global start
    values_classes=np.unique(pred, return_index=False, return_inverse=False, return_counts=False, axis=None)
    miss = missing_elements(values_classes,wrist_labels,Y)
    for i in miss: 
        if start==0:
            i=i
        else:
            i=i-1
        pred_binarized=np.hstack((pred_binarized[:,:i], np.zeros((pred_binarized.shape[0], 1)), pred_binarized[:,i:]))

    #Check if some class is missing in probabilities
    values_classes=np.unique(Y, return_index=False, return_inverse=False, return_counts=False, axis=None)
    columns=labels
    df = pd.DataFrame()
    j=0
    for i in  values_classes:
        if start==0:
            i=i
        else:
            i=i-1
        if int(i)>int(j):
            df.insert(int(j), column=columns[int(i)], value=proba[:,int(j)])
        else:
            df.insert(int(j), column=columns[int(i)], value=proba[:,int(i)])
        j=j+1

            
    miss = missing_elements(values_classes,columns, wrist_Y)
    
    #include missing probabilities
    for i in miss: 
        if start==0:
            i=i
        else:
            i=i-1
        df.insert(int(i), column=columns[int(i)], value=np.zeros(len(proba[:,int(0)])))

    proba=df.values
    return pred_binarized, proba, df


#Perfom the classification process for one stage of the model cascade for the secuential implementation
def classif_model(pipe,wrist_X,wrist_Y,log_loss_T,final_model):
    global n_samples
    global start
    score_selection=[]
    
    #Obtain the probability and prediction of an intance
    proba = cross_val_predict(pipe, wrist_X, wrist_Y, cv =kf, method='predict_proba')
    pred = cross_val_predict(pipe, wrist_X, wrist_Y, cv =kf)

    #Label binarization
    lb = LabelBinarizer2()
    wrist_Y_binarized = lb.fit_transform(wrist_Y)
    pred_binarized = lb.fit_transform(pred)

    #Check and add missing elements
    pred_binarized, proba, df = complete_arrays(wrist_Y,pred,pred_binarized,proba,wrist_labels)

    #Obtain log loss value
    ll_pred = multiclass_log_loss(pred_binarized, proba)

    #Create dataframe containing all the information about classified instances
    df3 = pd.DataFrame(pred,columns=['Pred_label'])
    df2 = pd.DataFrame(wrist_Y,columns=['True_label'])
    df = pd.DataFrame(proba,columns=labels)
    df=df.join(df2)
    df=df.join(df3)
    df5 =pd.DataFrame(ll_pred, columns=["LOG_LOSS"])
    df=df.join(df5)

    #Add new colum to the Dataframe containing the new predictions based on calculated threshold
    df['Error_thres'] = np.where(df['Pred_label'] == df['True_label'], '_', 'TRUE')

    #Compare the confidence threshold to obtain the pending and classified instances with 
    if final_model == False: #n-1 level of the cascade
        df_thres_NO_S=df[(df.LOG_LOSS <= log_loss_T)] #Clasificados
        df_pending_NO_S=df[(df.LOG_LOSS >= log_loss_T)] #Pending
        n_samples=((df_pending_NO_S.shape)[0])

        selection= df_thres_NO_S.shape[0]
        index_selection= df_pending_NO_S.index.values.tolist()
                
        errors_selection= df_thres_NO_S[(df_thres_NO_S.Error_thres == 'TRUE')].shape[0]
        score_selection= f1_score(df_thres_NO_S["True_label"].values, df_thres_NO_S["Pred_label"].values, average="macro")

        #check if the number of pending instances is enouch to perform the 5-CV process
        if len(df_thres_NO_S.index.values.tolist()) > 0:
            clasif_report= classification_report(df_thres_NO_S["True_label"].values, df_thres_NO_S["Pred_label"].values, digits=4)
        else:
            clasif_report=[]
            #print("Not enough instances to classify with model")
    else: #Last level of the cascade
        clasif_report=classification_report(df["True_label"].values, df["Pred_label"].values, digits=4)
        df_thres_NO_S=df.copy()
        errors_selection= df_thres_NO_S[(df_thres_NO_S.Error_thres == 'TRUE')].shape[0]
        score_selection= f1_score(df_thres_NO_S["True_label"].values, df_thres_NO_S["Pred_label"].values, average="macro")
        index_selection=[]
    return(index_selection,df_thres_NO_S,clasif_report,errors_selection,score_selection)



#Perfom the classification process for one stage of the model cascade for paralel implementation
def classif_model_paralel(pipe,wrist_X,wrist_Y,log_loss_T,final_model,model_2):
    global n_samples
    global index_selection_M1, index_selection_M2

    score_selection=[]
    
    #Obtain the probability and prediction of an intance
    proba = cross_val_predict(pipe, wrist_X, wrist_Y, cv =kf, method='predict_proba')
    pred = cross_val_predict(pipe, wrist_X, wrist_Y, cv =kf)

    #Label binarization
    lb = LabelBinarizer2()
    wrist_Y_binarized = lb.fit_transform(wrist_Y)
    pred_binarized = lb.fit_transform(pred)

    pred_binarized, proba, df = complete_arrays(wrist_Y,pred,pred_binarized,proba,wrist_labels)

    
    ll_pred = multiclass_log_loss(pred_binarized, proba)

    if model_2==True: #n-1 level
        pred=pred[index_selection_M1]
        proba=proba[index_selection_M1]
        ll_pred=ll_pred[index_selection_M1]
        wrist_Y=wrist_Y[index_selection_M1]
            
    elif final_model == True: #Final level cascade
        pred_selection=pred[index_selection_M1]
        pred=pred_selection[index_selection_M2]

        proba_selection=proba[index_selection_M1]
        proba=proba_selection[index_selection_M2]

        wrist_Y_selection=wrist_Y[index_selection_M1]
        wrist_Y=wrist_Y_selection[index_selection_M2]
 

        ll_pred_m3_selection=ll_pred[index_selection_M1]
        ll_pred=ll_pred_m3_selection[index_selection_M2]


    
    #Dataframe 
    df3 = pd.DataFrame(pred,columns=['Pred_label'])
    df2 = pd.DataFrame(wrist_Y,columns=['True_label'])
    df = pd.DataFrame(proba,columns=labels)
    df=df.join(df2)
    df=df.join(df3)
    df5 =pd.DataFrame(ll_pred, columns=["LOG_LOSS"])
    df=df.join(df5)

    #Add new colum to the Dataframe containing the new predictions based on calculated threshold
    df['Error_thres'] = np.where(df['Pred_label'] == df['True_label'], '_', 'TRUE')

    if final_model == False:
        df_thres_NO_S=df[(df.LOG_LOSS <= log_loss_T)] #Clasificados
        df_pending_NO_S=df[(df.LOG_LOSS >= log_loss_T)] #Pending
        n_samples=((df_pending_NO_S.shape)[0])
        selection= df_thres_NO_S.shape[0]
        index_selection= df_pending_NO_S.index.values.tolist()
        
        if model_2==False:
            #If model 1, save indexes
            index_selection_M1=df_pending_NO_S.index.values.tolist()
        else:
            #If model 2, save indexes
            index_selection_M2=df_pending_NO_S.index.values.tolist()
            
        errors_selection= df_thres_NO_S[(df_thres_NO_S.Error_thres == 'TRUE')].shape[0]
        score_selection= f1_score(df_thres_NO_S["True_label"].values, df_thres_NO_S["Pred_label"].values, average="macro")
        #check if the number of pending instances is enouch to perform the 5-CV process
        if len(df_thres_NO_S.index.values.tolist()) > 0:
            clasif_report= classification_report(df_thres_NO_S["True_label"].values, df_thres_NO_S["Pred_label"].values, digits=4)
            #print(clasif_report)
        else:
            classif_report=[]
            #print("Not enough instances to classify with model 2")
    else:
        clasif_report=classification_report(df["True_label"].values, df["Pred_label"].values, digits=4)
        df_thres_NO_S=df.copy()
        errors_selection= df_thres_NO_S[(df_thres_NO_S.Error_thres == 'TRUE')].shape[0]
        score_selection= f1_score(df_thres_NO_S["True_label"].values, df_thres_NO_S["Pred_label"].values, average="macro")
        index_selection=[]
        #print(clasif_report)
        

    return(index_selection,df_thres_NO_S,clasif_report,errors_selection,score_selection)



    





## Todas clasificadas - Secuencial


In [41]:
selection_M1_array = [] #Selection M1
errors_selection_M1_array = [] # Errors_selection M1
score_selection_M1_array = [] #score selection M1
selection_M2_array= [] #Selection M2
errors_selection_M2_array = [] # errorsSelection M2
score_selection_M2_array= [] #score selection M2
index_selection=[]
wrist_X_new=[]
wrist_Y_new=[]
wrist_X_selection=[]
wrist_Y_selection=[]
wrist_X_NO_S_new=[]
wrist_Y_NO_S_new=[]
originalclass = []
predictedclass = []
original_class = []
predicted_class = []
original_class_M2 = []
predicted_clas_M2s = []
original_class_M2 = []
predicted_class_M2 = []
score_final =[]
score_final_M2 =[]
final_selection=[]
final_selection_before_M3=[]
initial_scores=[]
initial_scores_M1=[]
initial_scores_M2=[]
initial_scores_M2_all=[]
initial_scores_M3=[]





print("__________________SECUENCIAL__________________\n")

print ('--------------------------------------------------------------')
print ('--- Number of Features Model 1: %0.0f Feat. - Model 2: %0.0f Feat. - Model 3: %0.0f Feat.   ---' % (n_features_m1, n_features_m2, n_features_m3))
print ('--------------------------------------------------------------')
print('\n')

for item, item_name in zip(alg_array, alg_array_names): 
    print ('########### ' + str(item_name) + ' ##########')

    for n in range(0,n_times):
        
        #define stratified 5-fold cross validation
        kf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=n)
        
        ###########################################################################################
        #--------------------------- MODEL 1 ------------------------------------------------------
        ###########################################################################################

        #Feature selection definition
        select_feature = SelectKBest(chi2, k=n_features_m1)
        
        #Create pipeline to perform every processing stage only for the training folds of the 5-CV
        pipe = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
        ('Feature selection',select_feature),
                     ('Algorithm',item)])
        
        #Select the data input depending on segments true or false
        X_data_M1=check_segments(segments_m1)

        #Obtain reference score for model 1
        scores_M1 = cross_val_score(pipe, X_data_M1, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M1.append(np.mean(scores_M1))

        #Results the fist level of the model cascade
        #df_thres_1 dataframe contains the classified instances of model 1
        index_selection, df_thres_1,clasif_report_M1,errors_selection_M1,score_selection_M1= classif_model(pipe, X_data_M1 ,wrist_Y, log_loss_T,final_model=False)
    
        #Classified instances X (for segmented and no segmented data)
        if segments_m2==True:
            wrist_X_selection_M1=wrist_X[index_selection]
        else:
            wrist_X_selection_M1=wrist_X_NO_S[index_selection] #pending 
            wrist_X_selection_M1_segments=wrist_X[index_selection]


        #Classified instances Y
        wrist_Y_selection_M1=wrist_Y[index_selection] 
        selection_M1=df_thres_1.shape[0] # Number of instances cassified
        
        #Create array for averaging results
        initial_scores_M1.append(np.mean(scores_M1)) #Reference M1 score array
        selection_M1_array.append(selection_M1) #Number of instances cassified M1 array
        errors_selection_M1_array.append(errors_selection_M1) #errors selection M1 array
        score_selection_M1_array.append(score_selection_M1) #F1 score selection M1 array

        
        ###########################################################################################
        #--------------------------- MODEL 2 ------------------------------------------------------ 
        ###########################################################################################

        #Feature selection definition
        select_feature_M2 = SelectKBest(chi2, k=n_features_m2)
        
        #Create pipeline to perform every processing stage only for the training folds of the 5-CV
        pipe2 = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
            ('Feature selection',select_feature_M2), ('Algorithm',item)])

        #Select the data input depending on segments true or false
        X_data_M2=check_segments(segments_m2)

        #Obtain reference score for model 2
        scores_M2 = cross_val_score(pipe2, X_data_M2, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M2.append(np.mean(scores_M2))

              
        # Check the number of remaining instances to avoid exceptions in the 5CV
        if ((n_splits*1.5 < n_samples)and ((check_CV_instances(wrist_Y_selection_M1)== False) or (n_samples > number_classes*2))):
            
            #Results the second level of the model cascade 
            #df_thres_2 dataframe contains the classified instances of model 2
            index_selection, df_thres_2,clasif_report_M2,errors_selection_M2,score_selection_M2= classif_model(pipe2, wrist_X_selection_M1,wrist_Y_selection_M1,log_loss_T_sel,final_model=False)
            

            if segments_m2==True: 
                wrist_X_selection_M2=wrist_X_selection_M1[index_selection] 
            else: 
                wrist_X_selection_M2=wrist_X_selection_M1_segments[index_selection] 

                
            #Classified instances
            wrist_Y_selection_M2=wrist_Y_selection_M1[index_selection] 
            selection_M2=df_thres_2.shape[0] #Number of instances cassified

            selection_M2_array.append(selection_M2) #Number of instances cassified M2 array
            errors_selection_M2_array.append(errors_selection_M2) #errors selection M2 array
            score_selection_M2_array.append(score_selection_M2) #F1 score selection M2 array
                
        else: #If number of remaining instances is not enough for 5-CV
            df_thres_2 = pd.DataFrame()
            if segments_m2==True: 
                wrist_X_selection_M2=wrist_X_selection_M1 #pending 
            else: 
                wrist_X_selection_M2=wrist_X_selection_M1_segments #pending
            wrist_Y_selection_M2=wrist_Y_selection_M1 #pending
            selection_M2=0
        
        #------------------ MODEL 1 + Model 2 --------------------

        #Join m1+m2 in one dataframe
        M2_all= [df_thres_1, df_thres_2]
        final_df_M2 = pd.concat(M2_all)
        selection_M2_all=final_df_M2.shape[0]

        #results of first two levels (m1 and m2) of the cascade
        errors_selection_final_M2= final_df_M2[(final_df_M2.Error_thres == 'TRUE')].shape[0]
        score_selection_final_M2= f1_score(final_df_M2["True_label"].values, final_df_M2["Pred_label"].values, average="macro")
        score_final_M2.append(score_selection_final_M2)


        ###########################################################################################
        #--------------------------- MODEL 3 ------------------------------------------------------ 
        ###########################################################################################



        #Feature selection definition
        select_feature_M3 = SelectKBest(chi2, k=n_features_m3)
        pipe3 = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
            ('Feature selection',select_feature_M3), ('Algorithm',item)])

        #Check segments model 3
        X_data_M3=check_segments(segments_m3)
        X_data_M3

        #Obtain reference score model 3
        scores_M3 = cross_val_score(pipe3, X_data_M3, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M3.append(np.mean(scores_M3))
              
        # Check the number of remaining instances to avoid exceptions in the 5CV
        if ((n_splits*1.5 < n_samples)and ((check_CV_instances(wrist_Y_selection_M2)== False) or (n_samples > number_classes*2))):
            
            #No treshold is applied at the last stage
            log_loss_T_sel_3=0
            
            #Results the third level of the model cascade 
            #df_thres_2 dataframe contains the classified instances of model 3
            index_selection, df_thres_3,clasif_report_M3,errors_selection_M3,score_selection_M3= classif_model(pipe3, wrist_X_selection_M2,wrist_Y_selection_M2,log_loss_T_sel,final_model=True)
            #Classified instances
            selection_M3=df_thres_3.shape[0]
        else:#If number of remaining instances is not enough for 5-CV
            #print("Number of pending instances not enough to perform Model3 classification")
            df_thres_3 = pd.DataFrame()



        ##########################################################################################
        #--------------------------- FINAL MODEL---------------------------------------------------
        ###########################################################################################

        #Summary of results for the whole cascade
        #Create dataframe containing all the classified instances
        final= [df_thres_1, df_thres_2, df_thres_3]
        final_df = pd.concat(final)
        
        #Classified instances
        selection_M3=df_thres_3.shape[0]
        selection_final= final_df.shape[0]
        
        #Errors
        errors_selection_final= final_df[(final_df.Error_thres == 'TRUE')].shape[0]
        
        #Final Score
        score_selection_final= f1_score(final_df["True_label"].values, final_df["Pred_label"].values, average="macro")
        score_final.append(score_selection_final)
        original_class.extend(final_df["True_label"].values)
        predicted_class.extend(final_df["Pred_label"].values)
        
        #Final selection adn errors
        final_sum= selection_M1 + selection_M2 + selection_M3
        final_sum_before_M3= selection_M1 + selection_M2
        final_selection.append(final_sum)
        final_selection_before_M3.append(final_sum_before_M3)

    #Classification report final cascade
    clasif_report_selection_final= classification_report(final_df["True_label"].values, final_df["Pred_label"].values,  digits=4) #target_names=wrist_labels_binary,
    clasif_report_selection_final= classification_report(original_class, predicted_class, digits=4, target_names=wrist_labels)#, target_names=wrist_labels_binary


    #------------SUMMARY MODEL 1
    print(" \n Model 1 \n")
    print(" \nInitial Mean F1_macro: " + str(np.mean(initial_scores_M1)))
    print("Mean Selection: " + str(np.mean(selection_M1_array)) + " - out of " +str(len(wrist_X)))
    print("Mean Number of errores selection: " + str(np.mean(errors_selection_M1_array)))
    print("Mean F1_macro selection: "+ str(np.mean(score_selection_M1_array)))
    print("Clasification report final M1 \n")
    print(clasif_report_M1)

    #------------SUMMARY MODEL 2
    print(" \n Model 2 \n")
    print(" \nInitial Mean F1_macro: " + str(np.mean(initial_scores_M2)))
    print("Mean Selection: " + str(np.mean(selection_M2_array)) + " out of " + str (len(wrist_X) - np.mean(selection_M1_array)))
    print("Mean Number of errores selection: " + str(np.mean(errors_selection_M2_array)))
    print("Mean F1_macro selection: "+ str(np.mean(score_selection_M2_array)))
    print("Mean F1_macro for M1 + M2: "+ str(np.mean(score_final_M2)))
    print("Clasification report final M2 \n")
    print(clasif_report_M2)

    #------------SUMMARY Final
    print("\n Final \n")
    print("\nFinal Mean F1_macro: " + str(np.mean(score_final)*100)+" - SD " + str(np.std(score_final)*100))
    #print('%.2f (SD %.3f)' % (np.mean(score_final)*100, np.std(score_final)*100))
    print("Classified: " + str(np.mean(final_selection)) + " /  Not classified " + str (len(wrist_X)- np.mean(final_selection)))
    print("Classified Before Final stage: " + str(np.mean(final_selection_before_M3)) + "- SD "+ str(np.std(final_selection_before_M3))+" /  Not classified " + 
          str (len(wrist_X)- np.mean(final_selection_before_M3)))
    
    print("Mean Number of errors selection: " + str(np.mean(errors_selection_M1_array)+np.mean(errors_selection_M2_array)))
    print("\n Clasification report final SELECTION \n")
    print(clasif_report_selection_final)
    

__________________SECUENCIAL__________________

--------------------------------------------------------------
--- Number of Features Model 1: 10 Feat. - Model 2: 50 Feat. - Model 3: 486 Feat.   ---
--------------------------------------------------------------


########### LG ##########
 
 Model 1 

 
Initial Mean F1_macro: 0.666539879844612
Mean Selection: 305.3 - out of 1000
Mean Number of errores selection: 6.1
Mean F1_macro selection: 0.623391531517089
Clasification report final M1 

              precision    recall  f1-score   support

         0.0     1.0000    0.9885    0.9942       262
         1.0     0.0000    0.0000    0.0000         2
         2.0     0.8000    1.0000    0.8889        20

    accuracy                         0.9824       284
   macro avg     0.6000    0.6628    0.6277       284
weighted avg     0.9789    0.9824    0.9798       284

 
 Model 2 

 
Initial Mean F1_macro: 0.7946448041983922
Mean Selection: 398.9 out of 694.7
Mean Number of errores selection

# # Todas classificadas Parale

In [48]:
print("________PARALEL_________\n")


index_selection_M1=[]
index_selection_M2=[]
index_selection=[]
wrist_X_new=[]
wrist_Y_new=[]
wrist_X_selection=[]
wrist_Y_selection=[]
wrist_X_NO_S_new=[]
wrist_Y_NO_S_new=[]
index_selection_M1=[]
index_selection_M2=[]
index_selection_M3=[]
originalclass = []
predictedclass = []
original_class = []
predicted_class = []
selection_M1_array = [] #Selection M1
errors_selection_M1_array = [] # Errors_selection M1
score_selection_M1_array = [] #score selection M1
selection_M1_array #selection
errors_selection_M1_array #errors_selection
score_selection_M1_array #F1 score selection
selection_M1_array=[]
errors_selection_M1_array=[]
score_selection_M1_array=[]
score_final_M2=[]
score_final =[]
final_selection=[]
initial_scores_M1=[]
initial_scores_M2=[]
initial_scores_M3=[]
final_selection_before_M3=[]
scores_M3=[]

labels=wrist_labels



print ('--------------------------------------------------------------')
print ('--- Number of Features Model 1: %0.0f Feat. - Model 2: %0.0f Feat. - Model 3: %0.0f Feat. ---' % (n_features_m1, n_features_m2,n_features_m3))
print ('--------------------------------------------------------------')
print('\n')

for item, item_name in zip(alg_array, alg_array_names): 
    print ('########### ' + str(item_name) + ' ##########')


    for n in range(0,n_times):
        
        #define stratified 5-fold cross validation
        kf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=n)


        ###########################################################################################
        #--------------------------- MODEL 1 ------------------------------------------------------
        ###########################################################################################

        #Feature selection definition
        select_feature = SelectKBest(chi2, k=n_features_m1)
        
        #Create pipeline to perform every processing stage only for the training folds of the 5-CV
        pipe = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
        ('Feature selection',select_feature),
                     ('Algorithm',item)])

        
        #Select the data input depending on segments true or false
        X_data_M1=check_segments(segments_m1)

        #Obtain reference score for model 1
        scores_M1 = cross_val_score(pipe, X_data_M1, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M1.append(np.mean(scores_M1))


        #Results the fist level of the model cascade
        #df_thres_1 dataframe contains the classified instances of model 1
        index_selection_M1, df_thres_1,clasif_report_M1,errors_selection_M1,score_selection_M1= classif_model_paralel(pipe, X_data_M1,wrist_Y, log_loss_T,final_model=False,model_2=False)
        
        #Classified instances X (for segmented and no segmented data)
        if segments_m2==True:
            wrist_X_selection_M1=wrist_X[index_selection_M1]
        else:
            wrist_X_selection_M1=wrist_X_NO_S[index_selection_M1] #pending 
            wrist_X_selection_M1_segments=wrist_X[index_selection_M2]


        #Classified instances
        wrist_Y_selection_M1=wrist_Y[index_selection_M1] #
        selection_M1=df_thres_1.shape[0] # Number of instances classified

        #Create array for averaging results
        initial_scores_M1.append(np.mean(scores_M1)) #Reference M1 score array
        selection_M1_array.append(selection_M1) #Number of instances cassified M1 array
        errors_selection_M1_array.append(errors_selection_M1) #errors selection M1 array
        score_selection_M1_array.append(score_selection_M1) #F1 score selection M1 array


        
        ###########################################################################################
        #--------------------------- MODEL 2 ------------------------------------------------------ 
        ###########################################################################################

        #Feature selection definition
        select_feature_M2 = SelectKBest(chi2, k=n_features_m2)
        
        #Create pipeline to perform every processing stage only for the training folds of the 5-CV
        pipe2 = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
            ('Feature selection',select_feature_M2), ('Algorithm',item)])

        #Select the data input depending on segments true or false
        X_data_M2=check_segments(segments_m2)
        
        #Obtain reference score for model 2
        scores_M2 = cross_val_score(pipe2, X_data_M2, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M2.append(np.mean(scores_M2))

        # Check the number of remaining instances to avoid exceptions in the 5CV
        if ((n_splits*1.5 < n_samples)and ((check_CV_instances(wrist_Y_selection_M1)== False) or (n_samples > number_classes*2))):
            
            #Results the second level of the model cascade 
            #df_thres_2 dataframe contains the classified instances of model 2
            index_selection_M2, df_thres_2,clasif_report_M2,errors_selection_M2,score_selection_M2= classif_model_paralel(pipe2, X_data_M2,wrist_Y,log_loss_T_sel,final_model=False,model_2=True)

            #Classified instances
            selection_M2=df_thres_2.shape[0]
            selection_M2_array.append(selection_M2) #Number of instances cassified
            wrist_Y_selection=wrist_Y[index_selection_M1]
            wrist_Y_selection_M2=wrist_Y_selection[index_selection_M2]
            wrist_X_selection=wrist_X[index_selection_M1]
            wrist_X_selection_M2=wrist_X_selection[index_selection_M2]
            
            errors_selection_M2_array.append(errors_selection_M2) #errors selection M2 array
            score_selection_M2_array.append(score_selection_M2) #F1 score selection M2 array


        else: #If number of remaining instances is not enough for 5-CV
            df_thres_2 = pd.DataFrame()
            selection_M2=0
            index_selection_M2 =index_selection_M1
            wrist_Y_selection_M2=wrist_Y_selection_M1

            
        #------------------ MODEL 1 + Model 2 --------------------
        
        #Join m1+m2 in one dataframe
        M2_all= [df_thres_1, df_thres_2]
        final_df_M2 = pd.concat(M2_all)
        selection_M2_all=final_df_M2.shape[0]

        #results of first two levels (m1 and m2) of the cascade
        errors_selection_final_M2= final_df_M2[(final_df_M2.Error_thres == 'TRUE')].shape[0]
        score_selection_final_M2= f1_score(final_df_M2["True_label"].values, final_df_M2["Pred_label"].values, average="macro")
        score_final_M2.append(score_selection_final_M2)



        ###########################################################################################
        #--------------------------- MODEL 3 ------------------------------------------------------ 
        ###########################################################################################
     
        
        #Feature selection definition
        select_feature_M3 = SelectKBest(chi2, k=n_features_m3)
        pipe3 = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
            ('Feature selection',select_feature_M3), ('Algorithm',item)])
        
        #Check segments model 3
        X_data_M3=check_segments(segments_m3)
        
        #Obtain reference score model 3
        scores_M3 = cross_val_score(pipe3, wrist_X, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M3.append(np.mean(scores_M3))


        # Check the number of remaining instances to avoid exceptions in the 5CV
        if ((n_splits*1.5 < n_samples)and ((check_CV_instances(wrist_Y_selection_M2)== False) or (n_samples > number_classes*2))):
            #No treshold is applied at the last stage
            log_loss_T_sel_3=0
            
            #Results the third level of the model cascade 
            #df_thres_2 dataframe contains the classified instances of model 3
            index_selection_M3, df_thres_3,clasif_report_M3,errors_selection_M3,score_selection_M3= classif_model_paralel(pipe3, X_data_M3,wrist_Y,log_loss_T_sel,final_model=True,model_2=False)
            #Classified instances
            selection_M3=df_thres_3.shape[0]
        else:#If number of remaining instances is not enough for 5-CV
            #print("Number of pending instances not enough to perform Model3 classification")
            df_thres_3 = pd.DataFrame()




        ##########################################################################################
        #--------------------------- FINAL MODEL---------------------------------------------------
        ###########################################################################################

        #Summary of results for the whole cascade
        #Create dataframe containing all the classified instances
        final= [df_thres_1, df_thres_2, df_thres_3]
        final_df = pd.concat(final)
        
        #Classified instances
        selection_M3=df_thres_3.shape[0]
        selection_final= final_df.shape[0]
        
        #Errors
        errors_selection_final= final_df[(final_df.Error_thres == 'TRUE')].shape[0]
        
        #Final Score
        score_selection_final= f1_score(final_df["True_label"].values, final_df["Pred_label"].values, average="macro")
        score_final.append(score_selection_final)
        original_class.extend(final_df["True_label"].values)
        predicted_class.extend(final_df["Pred_label"].values)

        #Final selection adn errors
        final_sum= selection_M1 + selection_M2 + selection_M3
        final_sum_before_M3= selection_M1 + selection_M2
        final_selection.append(final_sum)
        final_selection_before_M3.append(final_sum_before_M3)


    #Classification report final cascade
    clasif_report_selection_final= classification_report(final_df["True_label"].values, final_df["Pred_label"].values,  digits=4) #target_names=wrist_labels_binary,
    clasif_report_selection_final= classification_report(original_class, predicted_class, digits=4, target_names=labels)#, target_names=wrist_labels_binary



    print(" \n Model 1 \n")
    print(" \nInitial Mean F1_macro: " + str(np.mean(initial_scores_M1)))
    print("Mean Selection: " + str(np.mean(selection_M1_array)) + " - out of " +str(len(wrist_X)))
    print("Mean Number of errores selection: " + str(np.mean(errors_selection_M1_array)))
    print("Mean F1_macro selection: "+ str(np.mean(score_selection_M1_array)))
    print("Clasification report final M1 \n")
    print(clasif_report_M1)


    print(" \n Model 2 \n")
    print(" \nInitial Mean F1_macro: " + str(np.mean(initial_scores_M2)))
    print("Mean Selection: " + str(np.mean(selection_M2_array)) + " out of " + str (len(wrist_X) - np.mean(selection_M1_array)))
    print("Mean Number of errores selection: " + str(np.mean(errors_selection_M2_array)))
    print("Mean F1_macro selection: "+ str(np.mean(score_selection_M2_array)))
    print("Mean F1_macro for M1 + M2: "+ str(np.mean(score_final_M2)))
    print("Clasification report final M2 \n")
    print(clasif_report_M2)




    print("\n Final \n")
    print("\nFinal Mean F1_macro: " + str(np.mean(score_final)*100)+" - SD " + str(np.std(score_final)*100))
    #print('%.2f (SD %.3f)' % (np.mean(score_final)*100, np.std(score_final)*100))
    print("Classified: " + str(np.mean(final_selection)) + " /  Not classified " + str (len(wrist_X)- np.mean(final_selection)))
    print("Before M3: " + str(np.mean(final_selection_before_M3)) + "- SD "+ str(np.std(final_selection_before_M3))+" /  Not classified " + 
          str (len(wrist_X)- np.mean(final_selection_before_M3)))
    
    print("Mean Number of errors selection: " + str(np.mean(errors_selection_M1_array)+np.mean(errors_selection_M2_array)))
    print("Clasification report final SELECTION \n")
    print(clasif_report_selection_final)




________PARALEL_________

--------------------------------------------------------------
--- Number of Features Model 1: 10 Feat. - Model 2: 50 Feat. - Model 3: 486 Feat. ---
--------------------------------------------------------------


########### LG ##########
 
 Model 1 

 
Initial Mean F1_macro: 0.666539879844612
Mean Selection: 305.3 - out of 1000
Mean Number of errores selection: 6.1
Mean F1_macro selection: 0.623391531517089
Clasification report final M1 

              precision    recall  f1-score   support

         0.0     1.0000    0.9885    0.9942       262
         1.0     0.0000    0.0000    0.0000         2
         2.0     0.8000    1.0000    0.8889        20

    accuracy                         0.9824       284
   macro avg     0.6000    0.6628    0.6277       284
weighted avg     0.9789    0.9824    0.9798       284

 
 Model 2 

 
Initial Mean F1_macro: 0.7946448041983922
Mean Selection: 396.8 out of 694.7
Mean Number of errores selection: 44.6
Mean F1_macro sel

# Hybrid

In [55]:

print("________HYBRID_________\n")


index_selection_M1=[]
index_selection_M2=[]
index_selection=[]
wrist_X_new=[]
wrist_Y_new=[]
wrist_X_selection=[]
wrist_Y_selection=[]
wrist_X_NO_S_new=[]
wrist_Y_NO_S_new=[]

index_selection_M1=[]
index_selection_M2=[]
index_selection_M3=[]

originalclass = []
predictedclass = []
original_class = []
predicted_class = []




f1_score_array_LG_pre = []
f1_score_array_LG = []
n_error_LG = []



selection_M1_array = [] #Selection M1
errors_selection_M1_array = [] # Errors_selection M1
score_selection_M1_array = [] #score selection M1

selection_M1_array #selection
errors_selection_M1_array #errors_selection
score_selection_M1_array #F1 score selection



selection_M2_array= [] #Selection M2
errors_selection_M2_array = [] # errorsSelection M2
score_selection_M2_array= [] #score selection M2


score_final_M2=[]
score_final =[]
final_selection=[]
initial_scores_M1=[]
initial_scores_M2=[]
initial_scores_M3=[]

threshold_mean=[]

proba_NO_S=[]
final_selection_before_M3=[]
scores_M3=[]

labels=wrist_labels
def logloss(true_label, predicted, eps=1e-15):
  p = np.clip(predicted, eps, 1 - eps)
  if true_label == 1:
    return -np.log(p)
  else:
    return -np.log(1 - p)
    
def multiclass_log_loss(y_true, y_pred,eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1)



print ('--------------------------------------------------------------')
print ('--- Number of Features Model 1: %0.0f Feat. - Model 2: %0.0f Feat. - Model 3: %0.0f Feat. ---' % (n_features_m1, n_features_m2,n_features_m3))
print ('--------------------------------------------------------------')
print('\n')

for item, item_name in zip(alg_array, alg_array_names): 
    print ('########### ' + str(item_name) + ' ##########')



    for n in range(0,n_times):

        #define stratified 5-fold cross validation
        kf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=n)

        ###########################################################################################
        #--------------------------- MODEL 1 ------------------------------------------------------
        ###########################################################################################

        #Feature selection definition
        select_feature = SelectKBest(chi2, k=n_features_m1)
        
        #Create pipeline to perform every processing stage only for the training folds of the 5-CV
        pipe = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
        ('Feature selection',select_feature),
                     ('Algorithm',item)])

        #Select the data input depending on segments true or false
        X_data_M1=check_segments(segments_m1)

        #Obtain reference score for model 1
        scores_M1 = cross_val_score(pipe, X_data_M1, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M1.append(np.mean(scores_M1))


        #Results the fist level of the model cascade
        #df_thres_1 dataframe contains the classified instances of model 1
        index_selection_M1, df_thres_1,clasif_report_M1,errors_selection_M1,score_selection_M1= classif_model_paralel(pipe, X_data_M1,wrist_Y, log_loss_T,final_model=False,model_2=False)

        #Classified instances X (for segmented and no segmented data)
        if segments_m2==True:
            wrist_X_selection_M1=wrist_X[index_selection_M1]
        else:
            wrist_X_selection_M1=wrist_X_NO_S[index_selection_M1] #pending 
            wrist_X_selection_M1_segments=wrist_X[index_selection_M2]

        #Classified instances Y
        wrist_Y_selection_M1=wrist_Y[index_selection_M1] #pending
        selection_M1=df_thres_1.shape[0]

        #Create array for averaging results
        initial_scores_M1.append(np.mean(scores_M1))
        selection_M1_array.append(selection_M1) #selection
        errors_selection_M1_array.append(errors_selection_M1) #errors_selection
        score_selection_M1_array.append(score_selection_M1) #F1 score selection


        ###########################################################################################
        #--------------------------- MODEL 2 ------------------------------------------------------ 
        ###########################################################################################


        #Feature selection definition
        select_feature_M2 = SelectKBest(chi2, k=n_features_m2)
                
        #Create pipeline to perform every processing stage only for the training folds of the 5-CV
        pipe2 = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
            ('Feature selection',select_feature_M2), ('Algorithm',item)])

        #Select the data input depending on segments true or false
        X_data_M2=check_segments(segments_m2)
        
        #Obtain reference score for model 2
        scores_M2 = cross_val_score(pipe2, X_data_M2, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M2.append(np.mean(scores_M2))

        # Check the number of remaining instances to avoid exceptions in the 5CV
        if ((n_splits*1.5 < n_samples)and ((check_CV_instances(wrist_Y_selection_M1)== False) or (n_samples > number_classes*2))):
            
            #Results the second level of the model cascade 
            #df_thres_2 dataframe contains the classified instances of model 2
            index_selection_M2, df_thres_2,clasif_report_M2,errors_selection_M2,score_selection_M2= classif_model_paralel(pipe2, X_data_M2,wrist_Y,log_loss_T_sel,final_model=False,model_2=True)
            
            #Classified instances
            selection_M2=df_thres_2.shape[0]
            selection_M2_array.append(selection_M2) #Number of instances cassified
            errors_selection_M2_array.append(errors_selection_M2) #errors selection M2 array
            score_selection_M2_array.append(score_selection_M2) #F1 score selection M2 array
            
            #Y, X selection
            wrist_Y_selection=wrist_Y[index_selection_M1]
            wrist_Y_selection_M2=wrist_Y_selection[index_selection_M2]
            wrist_X_selection=wrist_X[index_selection_M1]
            wrist_X_selection_M2=wrist_X_selection[index_selection_M2]
            
        else:#If number of remaining instances is not enough for 5-CV
            df_thres_2 = pd.DataFrame()
            selection_M2=0
            index_selection_M2 =index_selection_M1
            wrist_Y_selection_M2=wrist_Y_selection_M1
            wrist_X_selection_M2=wrist_X_selection_M1
        #------------------ MODEL 1 + Model 2 --------------------

        #Join m1+m2 in one dataframe
        M2_all= [df_thres_1, df_thres_2]
        final_df_M2 = pd.concat(M2_all)
        selection_M2_all=final_df_M2.shape[0]

        #results of first two levels (m1 and m2) of the cascade
        errors_selection_final_M2= final_df_M2[(final_df_M2.Error_thres == 'TRUE')].shape[0]
        score_selection_final_M2= f1_score(final_df_M2["True_label"].values, final_df_M2["Pred_label"].values, average="macro")
        score_final_M2.append(score_selection_final_M2)



        ###########################################################################################
        #--------------------------- MODEL 3 ------------------------------------------------------ 
        ###########################################################################################


        #Feature selection definition
        select_feature_M3 = SelectKBest(chi2, k=n_features_m3)
        pipe3 = Pipeline([('Maxmin', preprocessing.MinMaxScaler()),
            ('Feature selection',select_feature_M3), ('Algorithm',item)])
        
        #Check segments model 3
        X_data_M3=check_segments(segments_m3)

        #Obtain reference score model 3
        scores_M3 = cross_val_score(pipe3, X_data_M3, wrist_Y, cv =kf, scoring = 'f1_macro')
        initial_scores_M3.append(np.mean(scores_M3))

        
        # Check the number of remaining instances to avoid exceptions in the 5CV
        if ((n_splits*2 < n_samples)and ((check_CV_instances(wrist_Y_selection_M2)== False) or (n_samples > number_classes*2))):
            #No treshold is applied at the last stage
            log_loss_T_sel_3=0
            #Results the third level of the model cascade 
            #df_thres_2 dataframe contains the classified instances of model 3
            index_selection, df_thres_3,clasif_report_M3,errors_selection_M2,score_selection_M2= classif_model(pipe3, wrist_X_selection_M2,wrist_Y_selection_M2,log_loss_T_sel,final_model=True)
            #Classified instances
            selection_M3=df_thres_3.shape[0]
        else:#If number of remaining instances is not enough for 5-CV
            df_thres_3 = pd.DataFrame()



        ##########################################################################################
        #--------------------------- FINAL MODEL---------------------------------------------------
        ###########################################################################################


        #Summary of results for the whole cascade
        #Create dataframe containing all the classified instances
        final= [df_thres_1, df_thres_2, df_thres_3]
        final_df = pd.concat(final)
        
        #Classified instances
        selection_M3=df_thres_3.shape[0]
        selection_final= final_df.shape[0]
        
        #Errors
        errors_selection_final= final_df[(final_df.Error_thres == 'TRUE')].shape[0]
        
        #Final Score
        score_selection_final= f1_score(final_df["True_label"].values, final_df["Pred_label"].values, average="macro")
        score_final.append(score_selection_final)
        original_class.extend(final_df["True_label"].values)
        predicted_class.extend(final_df["Pred_label"].values)


        #Final selection adn errors
        final_sum= selection_M1 + selection_M2 + selection_M3
        final_sum_before_M3= selection_M1 + selection_M2
        final_selection.append(final_sum)
        final_selection_before_M3.append(final_sum_before_M3)


    #Classification report final cascade
    clasif_report_selection_final= classification_report(final_df["True_label"].values, final_df["Pred_label"].values,  digits=4) #target_names=wrist_labels_binary,
    clasif_report_selection_final= classification_report(original_class, predicted_class, digits=4, target_names=labels)#, target_names=wrist_labels_binary




    print(" \n Model 1 \n")
    print(" \nInitial Mean F1_macro: " + str(np.mean(initial_scores_M1)))
    print("Mean Selection: " + str(np.mean(selection_M1_array)) + " - out of " +str(len(wrist_X)))
    print("Mean Number of errores selection: " + str(np.mean(errors_selection_M1_array)))
    print("Mean F1_macro selection: "+ str(np.mean(score_selection_M1_array)))
    print("Clasification report final M1 \n")
    print(clasif_report_M1)


    print(" \n Model 2 \n")
    print(" \nInitial Mean F1_macro: " + str(np.mean(initial_scores_M2)))
    print("Mean Selection: " + str(np.mean(selection_M2_array)) + " out of " + str (len(wrist_X) - np.mean(selection_M1_array)))
    print("Mean Number of errores selection: " + str(np.mean(errors_selection_M2_array)))
    print("Mean F1_macro selection: "+ str(np.mean(score_selection_M2_array)))
    print("Mean F1_macro for M1 + M2: "+ str(np.mean(score_final_M2)))
    print("Clasification report final M2 \n")
    print(clasif_report_M2)



    print("\n Final \n")
    print("\nFinal Mean F1_macro: " + str(np.mean(score_final)*100)+" - SD " + str(np.std(score_final)*100))
    #print('%.2f (SD %.3f)' % (np.mean(score_final)*100, np.std(score_final)*100))
    print("Classified: " + str(np.mean(final_selection)) + " /  Not classified " + str (len(wrist_X)- np.mean(final_selection)))
    print("Before M3: " + str(np.mean(final_selection_before_M3)) + "- SD "+ str(np.std(final_selection_before_M3))+" /  Not classified " + 
          str (len(wrist_X)- np.mean(final_selection_before_M3)))
    
    print("Mean Number of errors selection: " + str(np.mean(errors_selection_M1_array)+np.mean(errors_selection_M2_array)))
    print("Clasification report final SELECTION \n")
    print(clasif_report_selection_final)



________HYBRID_________

--------------------------------------------------------------
--- Number of Features Model 1: 10 Feat. - Model 2: 50 Feat. - Model 3: 486 Feat. ---
--------------------------------------------------------------


########### LG ##########
 
 Model 1 

 
Initial Mean F1_macro: 0.666539879844612
Mean Selection: 305.3 - out of 1000
Mean Number of errores selection: 6.1
Mean F1_macro selection: 0.623391531517089
Clasification report final M1 

              precision    recall  f1-score   support

         0.0     1.0000    0.9885    0.9942       262
         1.0     0.0000    0.0000    0.0000         2
         2.0     0.8000    1.0000    0.8889        20

    accuracy                         0.9824       284
   macro avg     0.6000    0.6628    0.6277       284
weighted avg     0.9789    0.9824    0.9798       284

 
 Model 2 

 
Initial Mean F1_macro: 0.7946448041983922
Mean Selection: 396.8 out of 694.7
Mean Number of errores selection: 44.6
Mean F1_macro sele